# Compute transforms from aspect solution DY/DZ to CHIPX/CHIPY

A key part of the process to apply dynamic ACA offsets to each observation is
computing the required DY/DZ aspect solution value to move the aimpoint to the
specified CHIPX/CHIPY coordinate.  This notebook computes these transforms
for each ACIS and HRC chip.

The final results are used in the `chandra_aca.drift` module.

This notebook was originally based on the `absolute_pointing_uncertainty` notebook
in this repository.

In [1]:
import argparse
import re
import time
from itertools import izip, cycle
import functools

import numpy as np
import Ska.DBI
from astropy.table import Table, vstack
from astropy.time import Time
import mica.archive.obspar
from mica.archive import asp_l1
import Ska.Shell
from Ska.quatutil import yagzag2radec
from Quaternion import Quat
from Ska.quatutil import radec2yagzag
from sherpa import ui

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Sherpa is too chatty so just turn off all logging (attempts to do this more selectively
# did not work on first try)
import logging
logging.disable(50)

In [3]:
# Nominal SIM focus and TSC positions.  This is run once with cell below and values
# are filled in to avoid repeating this exercise.  Originally use:
# sim_x_nom = {}
# sim_z_nom = {}
sim_x_nom = {
     'ACIS-I': -0.78090834371672724,
     'ACIS-S': -0.68282252473119054,
     'HRC-I': -1.0388663562382989,
     'HRC-S': -1.526339935833849}
sim_z_nom = {
     'ACIS-I': -233.58743446082869,
     'ACIS-S': -190.1400660498719,
     'HRC-I': 126.98297998998621,
     'HRC-S': 250.46603308020099}

In [4]:
if not sim_x_nom or not sim_z_nom:
    from Ska.DBI import DBI
    db = DBI(server='sybase', dbi='sybase', user='aca_read')
    dat = db.fetchall('select detector, sim_x, sim_z from obspar where obsid>12000 and obsid<18000')
    for det in ('ACIS-S', 'ACIS-I', 'HRC-S', 'HRC-I'):
        ok = dat['detector'] == det
        sim_x_nom[det] = np.median(dat['sim_x'][ok])
        sim_z_nom[det] = np.median(dat['sim_z'][ok])
    db.conn.close()
    from pprint import pprint
    pprint(sim_x_nom)
    pprint(sim_z_nom)

In [5]:
aimpoint_data_cache = {}
acis_pixscale = 0.492  # arcsec / pixel

def get_aimpoint_data(det):
    """
    Read aimpoint data provided by P. Zhao and define additional columns
    that make it easier to compare with fid light drift data.
    """
    if det not in aimpoint_data_cache:
        filename = 'optics_aimpoint/{}_ap_pos.rdb'.format(re.sub(r'-', '', det).lower())
        dat = Table.read(filename, format='ascii.rdb')
        y_off = dat['y_det_offset'] * 60 / acis_pixscale  # pix / arcmin
        z_off = dat['z_det_offset'] * 60 / acis_pixscale  # pix / arcmin

        aimpoint_data_cache[det] = dat
    return aimpoint_data_cache[det]    

In [6]:
def get_zero_offset_aimpoint_data(det, min_year=2010):
    """
    Return aimpoint data for observations with zero target offset.
    This simplifies the correlation of aspect solution and dmcoords
    results with the published POG plots (chapter 4) of aimpoint drift.
    """
    dat = get_aimpoint_data(det)
    ok = (dat['y_det_offset'] == 0) & (dat['z_det_offset'] == 0) & (dat['year'] > min_year)
    return dat[ok]

In [7]:
dats = get_zero_offset_aimpoint_data('ACIS-S', 2010)
dati = get_zero_offset_aimpoint_data('ACIS-I', 2010)
hrcs = get_zero_offset_aimpoint_data('HRC-S', 2010)
hrci = get_zero_offset_aimpoint_data('HRC-I', 2010)

observations = vstack([dats, dati, hrcs, hrci])

In [8]:
# Use the obspar for each obsid to fill in some additional
# columns.  Yag and Zag represent the local frame position
# (arcsec) of the target in the nominal frame.

noms = ('ra_nom', 'dec_nom', 'roll_nom')
for nom in noms:
    observations[nom] = 0.0
observations['yag'] = 0.0
observations['zag'] = 0.0
observations['ra_pnt'] = 0.0
observations['dec_pnt'] = 0.0
observations['detector'] = 'ACIS-S'

for obs in observations:
    obspar = mica.archive.obspar.get_obspar(obs['obsid'])
    for nom in noms:
        obs[nom] = obspar[nom]
    obs['ra_targ'] = obspar['ra_targ']
    obs['dec_targ'] = obspar['dec_targ']
    obs['ra_pnt'] = obspar['ra_pnt']
    obs['dec_pnt'] = obspar['dec_pnt']
    obs['detector'] = obspar['detector']
    q_nom = Quat([obs[nom] for nom in noms])
    obs['yag'], obs['zag'] = radec2yagzag(obspar['ra_targ'], obspar['dec_targ'], q_nom)

observations['yag'] *= 3600
observations['zag'] *= 3600

In [9]:
ok = observations['detector'] == 'HRC-I'
observations[ok][:5]

obsid,ra_targ,dec_targ,sim_z,sim_z_offset,y_det_offset,z_det_offset,dmt_chipx,dmt_chipy,ap_chipx,ap_chipy,year,ra_nom,dec_nom,roll_nom,yag,zag,ra_pnt,dec_pnt,detector
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,string48
10980.0,219.880833,-60.833611,126.98297999,0.00250890161531,0.0,0.0,7615.16,7876.76,7615.44,7877.04,2010.337018,219.889088992,-60.8351886006,0.206532993812,-14.4641698335,5.73062688131,219.889088992,-60.8351886006,HRC-I
10981.0,219.880833,-60.833611,126.98297999,0.00250890161531,0.0,0.0,7638.94,7834.59,7639.22,7834.87,2010.809946,219.871931629,-60.8332280321,196.582490367,-14.5737317164,5.77938669305,219.871931629,-60.8332280321,HRC-I
11808.0,10.684583,41.269278,126.98297999,0.00250890161531,0.0,0.0,7635.66,7878.25,7635.94,7878.53,2010.129265,10.6868073077,41.265356803,314.832594874,-14.2542449954,5.68438598704,10.6868073077,41.265356803,HRC-I
11809.0,10.684583,41.269278,126.98297999,0.00250890161531,0.0,0.0,7628.63,7855.13,7628.91,7855.41,2010.157781,10.6874732103,41.2655837895,322.265559712,-14.3240081833,5.73171324704,10.6874732103,41.2655837895,HRC-I
11819.0,125.489583,-43.004361,126.98297999,0.00250890161531,0.0,0.0,7629.57,7878.69,7629.85,7878.97,2010.611007,125.485042345,-43.0017377626,163.423426592,-14.1516322839,5.64101063257,125.485042345,-43.0017377626,HRC-I


### Use dmcoords to relate detector, dy, dz to chipx, chipy

This will be an approximation that applies over small displacements (of order 100 pixels).

In [10]:
ciaoenv = Ska.Shell.getenv('source /soft/ciao/bin/ciao.sh')

In [11]:
dmcoords_cmd = ['dmcoords', 'none',
                'asolfile=none',
                'detector="{detector}"',
                'opt=cel',
                'ra={ra}', 
                'dec={dec}',
                'celfmt=deg', 
                'ra_nom=0',
                'dec_nom=0',
                'roll_nom=0',  
                'ra_asp=")ra_nom"',
                'dec_asp=")dec_nom"',
                'roll_asp=")roll_nom"',      
                'sim="{simx} 0 {simz}"',
                'displace="0 {dy} {dz} 0 0 0"',       
                'verbose=0']
dmcoords_cmd = ' '.join(dmcoords_cmd)

In [12]:
ciaorun = functools.partial(Ska.Shell.bash, env=ciaoenv)

In [13]:
def dmcoords_chipx_chipy(det, dy, dz):
    """
    Get the dmcoords-computed chipx and chipy for given detector and
    aspect solution DY and DZ values.  NOTE: the ``dy`` and ``dz`` inputs
    to dmcoords are flipped in sign from the ASOL values.  Generally the
    ASOL DY/DZ are positive and dmcoord input values are negative.  This
    sign flip is handled *here*, so input to this is ASOL DY/DZ.
    
    :param det: detector (ACIS-S, ACIS-I, HRC-S, HRC-I)
    :param dy: aspect solution DY value (mm)
    :param dz: aspect solution DZ value (mm)
    """
    # See the absolute_pointing_uncertainty notebook in this repo for the
    # detailed derivation of this -15.5, 6.0 arcsec offset factor.  See the
    # cell below for the summary version.
    ra0, dec0 = -15.5 / 3600., 6.0 / 3600.
    ciaorun('punlearn dmcoords')
    cmd = dmcoords_cmd.format(ra=ra0, dec=dec0,
                                detector=(det if det.startswith('HRC') else 'ACIS'),
                                simx=sim_x_nom[det], simz=sim_z_nom[det],
                                dy=-dy, dz=-dz)
    ciaorun(cmd)
    return [float(x) for x in ciaorun('pget dmcoords chipx chipy chip_id')]

### Where does this -15.5, 6 arcsec offset come from?

It's basically the difference between two rotation matrices in our system:

- `ACA_MISALIGN`: MNC (HRMA optical axis) to ACA frame misalignment.
- `ODB_SI_ALIGN`: Misalignment used to transform from science target coordinates to ACA (PCAD) pointing direction that gets used on-board.

My recollection is that the fact that these are not the same is a relic of a decision during OAC, but I'm not entirely certain of that.

In [14]:
def get_dy_dz(obsid):
    """
    Get statistical summary data for aspect solution DY/DZ for ``obsid``.
    
    :param obsid: ObsID
    :returns: min_dy, median_dy, max_dy, min_dz, median_dz, max_dz (mm)
    """
    asolfiles = asp_l1.get_files(obsid=obsid, content=['ASPSOL'])
    asol = Table.read(asolfiles[0])
    min_dy, median_dy, max_dy = (np.min(asol['dy']), 
                              np.median(asol['dy']),
                              np.max(asol['dy']))
    min_dz, median_dz, max_dz = (np.min(asol['dz']), 
                              np.median(asol['dz']),
                              np.max(asol['dz']))
    return min_dy, median_dy, max_dy, min_dz, median_dz, max_dz                                 

### Do some exploration / validation of methods

In [15]:
# What are CHIPX/Y for HRC-I at DY/Z = 0, 0
dmcoords_chipx_chipy('HRC-I', 0, 0)

[-9831.388809890115, 7730.980954116741, 0.0]

#### Compare P. Zhao values to expected from dmcoords_chipx_chipy for ACIS-S and ACIS-I obsids

This is not incredibly independent since Zhao values also use dmcoords, but in a slightly different way.

In [16]:
dati['obsid', 'ap_chipx', 'ap_chipy'][:1]

obsid,ap_chipx,ap_chipy
float64,float64,float64
11613.0,933.95,997.64


In [17]:
min_dy, median_dy, max_dy, min_dz, median_dz, max_dz = get_dy_dz(11613)
print dmcoords_chipx_chipy(det='ACIS-I', dy=median_dy, dz=median_dz)

[934.1506010657735, 998.1400542414366, 3.0]


In [18]:
dats['obsid', 'ap_chipx', 'ap_chipy'][:1]

obsid,ap_chipx,ap_chipy
float64,float64,float64
12351.0,213.15,481.49


In [19]:
min_dy, median_dy, max_dy, min_dz, median_dz, max_dz = get_dy_dz(12351)
print dmcoords_chipx_chipy(det='ACIS-S', dy=median_dy, dz=median_dz)

[214.6393905516338, 480.61949999585, 7.0]


### DY and DZ correspond to adding values to Y_A and Z_A

<img src="http://cxc.harvard.edu/proposer/POG/html/images/simc.png">

### Find DY/DZ values corresponding to each ACIS-I chip

In [20]:
# The third number of the output is the chip_id
dmcoords_chipx_chipy(det='ACIS-I', dy=-10.0, dz=-10.0)

[342.318005673565, 545.2642470411855, 1.0]

In [21]:
dmcoords_chipx_chipy(det='ACIS-I', dy=10.0, dz=10.0)

[470.757080884857, 690.424018657772, 2.0]

In [22]:
dmcoords_chipx_chipy(det='ACIS-I', dy=10.0, dz=-10.0)

[682.2763973803094, 690.5173074492176, 0.0]

In [23]:
dmcoords_chipx_chipy(det='ACIS-I', dy=-10.0, dz=10.0)

[554.5064506202966, 544.7115493499787, 3.0]

### Use Sherpa and dmcoords to empirically determine the transformation DY/Z to CHIPX/Y for each chip

In [24]:
def dyz_to_chipxy(pars, x):
    """
    Define a function to transform from DY/Z to CHIPX/Y
    CHIPX, CHIPY = c0 + cyz * [DY, DZ]  (dot product)
    """
    c0x, c0y, cyx, czx, cyy, czy = pars
    c0 = np.array([[c0x],
                   [c0y]])
    cyz = np.array([[cyx, czx],
                    [cyy, czy]])
    x = np.array(x).reshape(-1, 2).transpose()
    y = c0 + cyz.dot(x)
    return y.transpose().flatten()

In [25]:
pars = [971.91, 963.07, 0.0, +41.74, -41.74, 0.0]
x = [-1, -1, 0, -1, -1, -2.0]
# x = [-1, -1]
print dyz_to_chipxy(pars, x)

[  930.17  1004.81   930.17   963.07   888.43  1004.81]


In [28]:
def fit_xform(det, chip_id, dy0, dz0):
    """
    Fit the transform model to 3 points forming an "L" on the chip around
    the ``dy0`` and ``dz0`` DY/Z values.
    
    :param det: detector (can be lower case)
    :param chip_id: numerical chip ID using dmcoord convention (ACIS=0 to 9, HRC=0 to 3)
    :param dy0: asol DY center value (mm)
    :param dz0: asol DZ center value (mm)
    
    :returns: sherpa fit_results() object
    """
    det = det.upper()
    x = []
    y = []
    for ddy, ddz in ((0, 0), (1, 0), (0, 1)):
        dy = dy0 + ddy
        dz = dz0 + ddz
        cx, cy, cid = dmcoords_chipx_chipy(det, dy, dz)
        if cid != chip_id:
            raise ValueError('Chip mismatch {} != {}'.format(cid, chip_id))
        x.extend([dy, dz])
        y.extend([cx, cy])
    ui.load_arrays(1, np.array(x), np.array(y))
    ui.load_user_model(dyz_to_chipxy, 'xform_mdl')
    ui.add_user_pars('xform_mdl', ['c0x', 'c0y', 'cyx', 'czx', 'cyy', 'czy'])
    ui.set_model(xform_mdl)
    ui.fit()
    
    return ui.get_fit_results()

In [29]:
fr = fit_xform('ACIS-I', 3, 0.0, 0.0)
fr.parvals

(971.91325933848623,
 963.17095452804222,
 -0.0010007356323315576,
 -41.741681607455625,
 41.741548160198832,
 -0.10439235761225273)

In [30]:
fr = fit_xform(det='HRC-I', chip_id=0, dy0=0.0, dz0=0.0)
fr.parvals

(-9831.3888098901152,
 7730.9809541167406,
 -109.98021295500075,
 109.9802129578411,
 109.98021295712488,
 109.98021295712579)

### Do the fitting for each chip on ACIS and (eventually) HRC

In [31]:
det_ids = (('acis-i', 3, 0, 0),
          ('acis-i', 0, 10, -10),
          ('acis-i', 1, -10, -10),
          ('acis-i', 2, 10, 10),
          ('acis-s', 7, 0, 0),
          ('acis-s', 6, 25, 0),
          ('acis-s', 5, 50, 0),
          ('acis-s', 4, 75, 0),
          ('acis-s', 8, -25, 0),
          ('acis-s', 9, -50, 0))

In [32]:
xforms = {}

In [33]:
for det, chip_id, dy0, dz0 in det_ids:
    key = det.upper(), chip_id
    if key in xforms:
        continue
    print(key)
    fr = fit_xform(det, chip_id, dy0, dz0)
    xforms[key] = fr.parvals

('ACIS-I', 3)
('ACIS-I', 0)
('ACIS-I', 1)
('ACIS-I', 2)
('ACIS-S', 7)
('ACIS-S', 6)
('ACIS-S', 5)
('ACIS-S', 4)
('ACIS-S', 8)
('ACIS-S', 9)


### Compare model prediction to actual

In [34]:
print dyz_to_chipxy(xforms['ACIS-I', 1], [-10, -10])
print dmcoords_chipx_chipy(det='ACIS-I', dy=-10.0, dz=-10.0)

[ 342.31800567  545.26424704]
[342.318005673565, 545.2642470411855, 1.0]


In [35]:
print dyz_to_chipxy(xforms['ACIS-S', 7], [0, 0])
print dmcoords_chipx_chipy(det='ACIS-S', dy=0.0, dz=0.0)

[ 252.27465647  519.86245656]
[252.2746564698478, 519.8624565623606, 7.0]


### Make a nicer format to copy into chandra_aca.drift module

In [36]:
xforms_out = {}
for key, val in xforms.items():
    vals = [round(x, 3) for x in val]
    xforms_out[key] = {'c0': [vals[0], vals[1]],
                      'cyz': [[vals[2], vals[3]],
                              [vals[4], vals[5]]]}

In [37]:
from pprint import pprint
pprint(xforms_out)

{('ACIS-I', 0): {'c0': [1099.688, 1108.985],
                 'cyz': [[0.001, 41.742], [-41.742, 0.105]]},
 ('ACIS-I', 1): {'c0': [-75.09, 963.721],
                 'cyz': [[0.001, -41.742], [41.741, 0.104]]},
 ('ACIS-I', 2): {'c0': [53.35, 1108.889],
                 'cyz': [[-0.001, 41.742], [-41.742, -0.105]]},
 ('ACIS-I', 3): {'c0': [971.913, 963.171],
                 'cyz': [[-0.001, -41.742], [41.742, -0.104]]},
 ('ACIS-S', 4): {'c0': [3383.032, 520.069],
                 'cyz': [[-41.695, -0.021], [0.021, -41.689]]},
 ('ACIS-S', 5): {'c0': [2339.627, 519.194],
                 'cyz': [[-41.691, -0.031], [0.037, -41.689]]},
 ('ACIS-S', 6): {'c0': [1296.718, 519.577],
                 'cyz': [[-41.69, -0.031], [0.036, -41.689]]},
 ('ACIS-S', 7): {'c0': [252.275, 519.862],
                 'cyz': [[-41.689, -0.022], [0.022, -41.689]]},
 ('ACIS-S', 8): {'c0': [-789.536, 519.829],
                 'cyz': [[-41.69, -0.02], [0.02, -41.689]]},
 ('ACIS-S', 9): {'c0': [-1831.755, 519.59